# LLM | Prompt ENG-1
- ollama download and install
- pip install ollama
- ollama run exaone3.5 (LG AI Research 개발) -> 시간이 오래걸림

In [12]:
# 모델 불러오기
import ollama

# 프롬포트 설정
prompt = """
You are a rehabilitation medicine doctor and have a level of fitness knowledge.
You have to speak in only Korean.
Don't use Chinese characters.
"""

# 채팅 응답 설정
chat_history = [
    {
        'system': prompt,
        'role': 'user',
        'content': '올바른 스쿼트 자세를 위해서 가장 중요한 스트레칭은 무엇일까?',
    }
]

# 응답
response = ollama.chat(model='exaone3.5', messages=chat_history)
print(response['message']['content'])

스쿼트 자세를 올바르게 유지하기 위해 가장 중요한 스트레칭들은 다음과 같습니다:

1. **하체 스트레칭**:
   - **허벅지 스트레칭 (Hamstring Stretch)**: 허벅지 뒷부분인 햄스트링 근육을 스트레칭하여 무릎이 과도하게 앞으로 치우치지 않도록 도와줍니다.
   - **허벅지 내측 스트레칭 (Hip Flexor Stretch)**: 허벅지 안쪽 근육을 스트레칭하여 허리와 엉덩이의 불균형을 줄여줍니다. 이는 스쿼트 동작 중 균형을 잡는 데 중요합니다.
   - **허벅지 외측 스트레칭 (Glute Stretch)**: 허벅지 바깥쪽 근육을 스트레칭하여 균형 잡힌 하체 움직임을 돕습니다.

2. **코어 및 척추 스트레칭**:
   - **척추 스트레칭**: 허리와 등 근육을 스트레칭하여 척추의 유연성을 높이고, 스쿼트 동작 중 올바른 자세를 유지하는 데 도움이 됩니다.
   - **복부 스트레칭**: 코어 근육을 강화하고 유연성을 향상시켜 척추를 지지하고 균형을 유지하는 데 도움이 됩니다.

이러한 스트레칭들을 꾸준히 수행하면 스쿼트 동작 중에 더 안정적이고 효과적인 자세를 유지하는 데 큰 도움이 됩니다. 스트레칭 전에는 항상 준비운동을 해주고, 스트레칭 시 과도한 통증을 느끼지 않는 범위 내에서 진행하는 것이 중요합니다.


# LLM | Prompt ENG-2

In [1]:
from collections import Counter
import json

In [2]:
# 데이터 로드 클래스 설계
class PoseAnalyzer():
    def __init__(self, input_path: str):
        self.input_path = input_path
        self.data = self.load_data()

    def load_data(self):
        with open(self.input_path, 'r', encoding='utf-8') as f:
            return json.load(f)
        
    def get_all_frames(self):
        return self.data

In [3]:
# 분석 데이터 경로
input_path = "../../LJH/output_json/front_pose_angle_eval.json"

pose_analyzer = PoseAnalyzer(input_path)
pose_analyzer.data

{'frame_000.json': {'left_hip_angle_diff': 5.09,
  'right_hip_angle_diff': 3.59,
  'left_knee_angle_diff': 0.65,
  'right_knee_angle_diff': 2.41,
  'pass_left_hip': 1,
  'pass_right_hip': 1,
  'pass_left_knee': 1,
  'pass_right_knee': 1,
  'cosine_similarity': 0.9999,
  'failed_parts': []},
 'frame_001.json': {'left_hip_angle_diff': 5.44,
  'right_hip_angle_diff': 3.49,
  'left_knee_angle_diff': 1.32,
  'right_knee_angle_diff': 3.19,
  'pass_left_hip': 1,
  'pass_right_hip': 1,
  'pass_left_knee': 1,
  'pass_right_knee': 1,
  'cosine_similarity': 0.9998,
  'failed_parts': []},
 'frame_002.json': {'left_hip_angle_diff': 5.41,
  'right_hip_angle_diff': 3.49,
  'left_knee_angle_diff': 1.32,
  'right_knee_angle_diff': 2.51,
  'pass_left_hip': 1,
  'pass_right_hip': 1,
  'pass_left_knee': 1,
  'pass_right_knee': 1,
  'cosine_similarity': 0.9998,
  'failed_parts': []},
 'frame_003.json': {'left_hip_angle_diff': 4.09,
  'right_hip_angle_diff': 2.39,
  'left_knee_angle_diff': 0.57,
  'right_kn

In [4]:
# 각도 분석 클래스 설계
class PoseEvaluator:
    def __init__(self, pose_analyzer: PoseAnalyzer, threshold: float = 15.0):
        self.data = pose_analyzer.get_all_frames()
        self.threshold = threshold
    
    def evaluate(self) -> dict:
        failed_frames = {}

        for frame_name, frame_data in self.data.items():
            failed_joints = []

            # *_angle_diff 키만 필터링
            for key, value in frame_data.items():
                if key.endswith('_angle_diff') and abs(value) >= self.threshold:
                    failed_joints.append(key)

            if failed_joints:
                failed_frames[frame_name] = failed_joints

        return failed_frames

In [5]:
pose_evaluate = PoseEvaluator(pose_analyzer=pose_analyzer)
failed_results = pose_evaluate.evaluate()

In [6]:
def summarize_failed_joints(failed_dict: dict) -> Counter:
    joint_counter = Counter()

    for joints in failed_dict.values():
        for joint in joints:
            joint_counter[joint] += 1
    
    return joint_counter

In [7]:
# 예시 실행
failed_joint_counts = summarize_failed_joints(failed_results)

for joint, count in failed_joint_counts.most_common():
    print(f"{joint}: {count}회")

right_knee_angle_diff: 39회
left_hip_angle_diff: 38회
left_knee_angle_diff: 30회
right_hip_angle_diff: 28회


In [8]:
def generate_summary_prompt(joint_counter: Counter) -> str:
    part_names = {
        "left_hip_angle_diff": "왼쪽 고관절",
        "right_hip_angle_diff": "오른쪽 고관절",
        "left_knee_angle_diff": "왼쪽 무릎",
        "right_knee_angle_diff": "오른쪽 무릎"
    }

    lines = []
    for joint, count in joint_counter.items():
        name = part_names.get(joint, joint)
        lines.append(f"- {name}: {count}회")
    
    prompt = (
        "운동 영상에서 다음 부위에 자주 문제가 발생했습니다:\n"
        f"{chr(10).join(lines)}\n\n"
        "이러한 문제가 왜 발생할 수 있는지, 그리고 어떻게 개선하면 좋을지 운동 전문가 입장에서 설명해 주세요."
    )
    return prompt

In [ ]:
prompt = generate_summary_prompt(failed_joint_counts)
response = get_llm